In [135]:
#If you run this yourself, make sure all libraries are installed and the csv file is in the same folder as the notebook
#import libraries
import numpy as np
import pandas as pd
import matplotlib as mlp
from matplotlib import pyplot as plt
mlp.rcParams['figure.dpi'] = 300
import scipy.stats
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

In [136]:
#import human graded data as pandas df
HG_EM = pd.read_csv("EM.csv") 
HG_QM = pd.read_csv("QM.csv") 
HG_CM = pd.read_csv("CM.csv") 
#import no markscheme data as pandas df
NMS_EM = pd.read_csv("Claude_EM_NMS.csv") 
NMS_QM = pd.read_csv("Claude_QM_NMS.csv")
NMS_CM = pd.read_csv("Claude_CM_NMS.csv")
#import markscheme data as pandas df
MS_EM = pd.read_csv("Claude_EM_MS.csv") 
MS_QM = pd.read_csv("Claude_QM_MS.csv")
MS_CM = pd.read_csv("Claude_CM_MS.csv")

In [137]:
#Examine the human graded data
HG_EM

,Question,Total Marks,M1_Solution 1:,M1_Solution 2:,M1_Solution 3:,M2_Solution 1,M2_Solution 2:,M2_Solution 3:
0,1,8,6,8,5,7,8,7
1,2,5,4,5,5,4,4,4
2,3,14,6,4,4,5,4,5
3,4,11,10,5,9,9,6,9
4,5,11,1,1,1,1,1,1
5,6,5,4,5,4,4,4,4
6,7,6,4,4,4,6,6,4
7,8,6,6,2,4,6,1,4
8,9,15,11,8,8,13,11,11
9,10,7,1,4,3,2,3,3


This seems alright, so we'll leave it as is for now. We don't need to make the initial changes as pandas does that for us. The code should work in either case. Now we need to average the solution grades. 

In [138]:
sol_1_cols = []
sol_2_cols = []
sol_3_cols = []
for column in HG_EM: 
    if 'Solution 1' in column:
        sol_1_cols.append(column)
    if 'Solution 2' in column:
        sol_2_cols.append(column)
    if 'Solution 3' in column:
        sol_3_cols.append(column)
HG = [HG_EM, HG_CM, HG_QM]
for df in HG:
    df['Solution 1'] = df[sol_1_cols].mean(axis = 1)
    df['Solution 2'] = df[sol_2_cols].mean(axis = 1)
    df['Solution 3'] = df[sol_3_cols].mean(axis = 1)
    df['Solution 1 Norm'] = df['Solution 1']/df['Total Marks']
    df['Solution 2 Norm'] = df['Solution 2']/df['Total Marks']
    df['Solution 3 Norm'] = df['Solution 3']/df['Total Marks']
    df['Solution 1 Norm Std'] = df[sol_1_cols].std(axis = 1)/df['Total Marks']
    df['Solution 2 Norm Std'] = df[sol_2_cols].std(axis = 1)/df['Total Marks']
    df['Solution 3 Norm Std'] = df[sol_3_cols].std(axis = 1)/df['Total Marks']
    df.drop(df.columns[[2, 3, 4, 5, 6, 7]], axis=1, inplace=True)    

In [139]:
HG_QM

,Question,Total Marks,Solution 1,Solution 2,Solution 3,Solution 1 Norm,Solution 2 Norm,Solution 3 Norm,Solution 1 Norm Std,Solution 2 Norm Std,Solution 3 Norm Std
0,1,5,4.0,4.5,4.0,0.800000,0.900000,0.800000,0.000000,0.141421,0.000000
1,2,4,4.0,3.5,3.5,1.000000,0.875000,0.875000,0.000000,0.176777,0.176777
2,3,9,9.0,5.5,9.0,1.000000,0.611111,1.000000,0.000000,0.078567,0.000000
3,4,20,15.0,11.0,13.5,0.750000,0.550000,0.675000,0.000000,0.141421,0.176777
4,5,7,5.0,4.0,5.0,0.714286,0.571429,0.714286,0.202031,0.202031,0.202031
5,6,17,10.0,9.5,8.5,0.588235,0.558824,0.500000,0.083189,0.041595,0.041595
6,7,8,6.0,7.5,7.0,0.750000,0.937500,0.875000,0.176777,0.088388,0.176777
7,8,9,8.0,8.5,8.5,0.888889,0.944444,0.944444,0.000000,0.078567,0.078567
8,9,17,11.5,12.5,12.0,0.676471,0.735294,0.705882,0.041595,0.041595,0.000000
9,10,16,5.0,3.0,2.0,0.312500,0.187500,0.125000,0.176777,0.000000,0.088388


In [140]:
sol_1_cols = []
sol_2_cols = []
sol_3_cols = []
for column in HG_EM: 
    if 'Solution 1' in column:
        sol_1_cols.append(column)
    if 'Solution 2' in column:
        sol_2_cols.append(column)
    if 'Solution 3' in column:
        sol_3_cols.append(column)

# Iterate through each DataFrame in the HG list
for i in range(len(HG)):
    df = HG[i]
    df1 = df.copy()
    df2 = df.copy()
    df3 = df.copy()
    
    # Drop the columns not related to each solution
    df1.drop(sol_2_cols + sol_3_cols, axis=1, inplace=True)
    df2.drop(sol_1_cols + sol_3_cols, axis=1, inplace=True)
    df3.drop(sol_1_cols + sol_2_cols, axis=1, inplace=True)
    
    # Rename the columns
    df1.columns = ['Questions', 'Total Marks', 'Marks', 'Norm', 'Norm Std']
    df2.columns = ['Questions', 'Total Marks', 'Marks', 'Norm', 'Norm Std']
    df3.columns = ['Questions', 'Total Marks', 'Marks', 'Norm', 'Norm Std']
    
    # Concatenate the DataFrames
    concatenated_df = pd.concat([df1, df2, df3], ignore_index=True, axis=0)
    
    # Assign the concatenated DataFrame back to the original list
    HG[i] = concatenated_df

In [141]:
HG_EM = HG[0]
HG_CM = HG[1]
HG_QM = HG[2]

In [142]:
#Examine the self graded data
NMS_EM

,Question,Total Marks,Unnamed: 2,Solution 1:,Solution 2:,Solution 3:,Unnamed: 6,Solution 1:.1,Solution 2:.1,Solution 3:.1,...,Solution 2:.2,Solution 3:.2,Unnamed: 14,Solution 1:.3,Solution 2:.3,Solution 3:.3,Unnamed: 18,Solution 1:.4,Solution 2:.4,Solution 3:.4
0,1,8,NaN,8,7,7,NaN,8,7,8,...,7,8,NaN,8,7,7,NaN,8,7,7
1,2,5,NaN,4,3,5,NaN,5,4,5,...,4,5,NaN,5,4,5,NaN,5,4,5
2,3,14,NaN,13,14,12,NaN,13,14,12,...,13,14,NaN,12,13,14,NaN,12,13,14
3,4,11,NaN,10,5,10,NaN,10,7,10,...,6,10,NaN,10,6,10,NaN,11,6,10
4,5,11,NaN,8,9,11,NaN,8,8,11,...,8,11,NaN,9,8,11,NaN,8,9,11
5,6,5,NaN,4,5,5,NaN,5,5,5,...,5,5,NaN,5,5,5,NaN,5,5,5
6,7,6,NaN,6,5,5,NaN,6,5,5,...,5,5,NaN,6,5,5,NaN,6,5,5
7,8,6,NaN,5,3,6,NaN,6,3,5,...,3,5,NaN,5,3,6,NaN,6,3,5
8,9,15,NaN,13,14,15,NaN,15,15,14,...,14,15,NaN,15,15,13,NaN,13,15,14
9,10,7,NaN,4,5,6,NaN,4,5,6,...,5,6,NaN,4,5,6,NaN,4,5,6


Couple things need to be changed. Index vs Question is fine. We need to removed all unnamed 

In [130]:
(np.linspace(2,16,15))

array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16.])

In [143]:
SG = [NMS_EM, NMS_CM, NMS_QM, MS_EM, MS_CM, MS_QM]

sol_1_cols = []
sol_2_cols = []
sol_3_cols = []

#Seperating Solution1,2,3 columns
for column in NMS_EM: 
    if 'Solution 1' in column:
        sol_1_cols.append(column)
    if 'Solution 2' in column:
        sol_2_cols.append(column)
    if 'Solution 3' in column:
        sol_3_cols.append(column)

for df in SG:
    for column in df:
        if 'Unnamed' in column:
            df.drop(column, axis = 1, inplace = True)
    df['Solution 1'] = df[sol_1_cols].mean(axis = 1)
    df['Solution 2'] = df[sol_2_cols].mean(axis = 1)
    df['Solution 3'] = df[sol_3_cols].mean(axis = 1)
    df['Solution 1 Norm'] = df['Solution 1']/df['Total Marks']
    df['Solution 2 Norm'] = df['Solution 2']/df['Total Marks']
    df['Solution 3 Norm'] = df['Solution 3']/df['Total Marks']
    df['Solution 1 Norm Std'] = df[sol_1_cols].std(axis = 1)/df['Total Marks']
    df['Solution 2 Norm Std'] = df[sol_2_cols].std(axis = 1)/df['Total Marks']
    df['Solution 3 Norm Std'] = df[sol_3_cols].std(axis = 1)/df['Total Marks']
    df.drop(df.columns[[2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
       15, 16]], axis=1, inplace=True)

In [144]:
NMS_EM

,Question,Total Marks,Solution 1,Solution 2,Solution 3,Solution 1 Norm,Solution 2 Norm,Solution 3 Norm,Solution 1 Norm Std,Solution 2 Norm Std,Solution 3 Norm Std
0,1,8,8.0,7.0,7.4,1.000000,0.875000,0.925000,0.000000,0.000000,0.068465
1,2,5,4.8,3.8,5.0,0.960000,0.760000,1.000000,0.089443,0.089443,0.000000
2,3,14,12.4,13.4,13.2,0.885714,0.957143,0.942857,0.039123,0.039123,0.078246
3,4,11,10.2,6.0,10.0,0.927273,0.545455,0.909091,0.040656,0.064282,0.000000
4,5,11,8.4,8.4,11.0,0.763636,0.763636,1.000000,0.049793,0.049793,0.000000
5,6,5,4.8,5.0,5.0,0.960000,1.000000,1.000000,0.089443,0.000000,0.000000
6,7,6,6.0,5.0,5.0,1.000000,0.833333,0.833333,0.000000,0.000000,0.000000
7,8,6,5.6,3.0,5.4,0.933333,0.500000,0.900000,0.091287,0.000000,0.091287
8,9,15,13.8,14.6,14.2,0.920000,0.973333,0.946667,0.073030,0.036515,0.055777
9,10,7,4.0,5.0,6.0,0.571429,0.714286,0.857143,0.000000,0.000000,0.000000


In [145]:
sol_1_cols = []
sol_2_cols = []
sol_3_cols = []
for column in NMS_EM: 
    if 'Solution 1' in column:
        sol_1_cols.append(column)
    if 'Solution 2' in column:
        sol_2_cols.append(column)
    if 'Solution 3' in column:
        sol_3_cols.append(column)

# Iterate through each DataFrame in the SG list
for i in range(len(SG)):
    df = SG[i]
    df1 = df.copy()
    df2 = df.copy()
    df3 = df.copy()
    
    # Drop the columns not related to each solution
    df1.drop(sol_2_cols + sol_3_cols, axis=1, inplace=True)
    df2.drop(sol_1_cols + sol_3_cols, axis=1, inplace=True)
    df3.drop(sol_1_cols + sol_2_cols, axis=1, inplace=True)
    
    # Rename the columns
    df1.columns = ['Questions', 'Total Marks', 'Marks', 'Norm', 'Norm Std']
    df2.columns = ['Questions', 'Total Marks', 'Marks', 'Norm', 'Norm Std']
    df3.columns = ['Questions', 'Total Marks', 'Marks', 'Norm', 'Norm Std']
    
    # Concatenate the DataFrames
    concatenated_df = pd.concat([df1, df2, df3], ignore_index=True, axis=0)
    
    # Assign the concatenated DataFrame back to the original list
    SG[i] = concatenated_df

In [146]:
NMS_EM = SG[0]
NMS_CM = SG[1]
NMS_QM = SG[2]
MS_EM = SG[3]
MS_CM = SG[4]
MS_QM = SG[5] 

In [ ]:
Test Github